In [45]:
!nvidia-smi

Tue Oct 27 10:52:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [46]:
%cd "/content/drive/My Drive/Fashion-MNIST"
# Import các package cần dùng
import numpy as np
import pandas as pd
import tensorflow as tf
import time

/content/drive/My Drive/Fashion-MNIST


In [47]:
# Load dữ liệu
training_set = pd.read_csv("./Dataset/fashion-mnist_train.csv").to_numpy()
test_set = pd.read_csv("./Dataset/fashion-mnist_test.csv").to_numpy()

# Tách label và data riêng cho tập train và test
X_train = training_set[:,1:]
Y_train = training_set[:,0]

X_test = test_set[:,1:]
Y_test = test_set[:,0]

# Chuyển dữ liệu sang dạng float32
X_train.astype(np.float32)
Y_train.astype(np.float32)
X_test.astype(np.float32)
Y_test.astype(np.float32)

array([0., 1., 2., ..., 8., 8., 1.], dtype=float32)

In [48]:
from sklearn.svm import SVC
start = time.time()

classifier = SVC(kernel='rbf', random_state=1, verbose=True)
classifier.fit(X_train, Y_train)

end_train = time.time()

Y_pred = classifier.predict(X_test)

end_pred = time.time()

acc = Y_pred == Y_test
print('\nTraining time: {:.0f} seconds'.format(end_train - start))
print('\nPrediction time: {:.0f} seconds'.format(end_pred - end_train))
print('\nAccuracy: ', acc.sum()/acc.shape[0])

[LibSVM]
Training time: 667 seconds

Prediction time: 235 seconds

Accuracy:  0.8921


In [49]:
from sklearn.decomposition import PCA
# Thực hiện PCA giảm từ 784 chiều (D) xuống 300 chiều (K)
pca = PCA(300)
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=300, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [50]:
# Ma trận không gian con thu được sau khi thực hiện PCA
U = pca.components_
U.shape # Shape: K x D

(300, 784)

In [51]:
# Tính toán giá trị của tập dữ liệu trên không gian con thu được
X_bar = X_train - pca.mean_ # Shape: N x D
Z = U.dot(X_bar.T) # Shape: K x D

Z.shape

(300, 60000)

In [52]:
# Chuyển tập dữ liệu về dạng hàng
X_train_PCA = Z.T

In [53]:
# Thực hiện tương tự với tập test
X_test_PCA = U.dot((X_test - pca.mean_).T).T
X_test_PCA.shape

(10000, 300)

In [54]:
from sklearn.svm import SVC

start = time.time()

classifier = SVC(kernel='rbf', random_state=1, verbose=True)

classifier.fit(X_train_PCA, Y_train)

end_train = time.time()

Y_pred = classifier.predict(X_test_PCA)

end_pred = time.time()

acc = Y_pred == Y_test

print('\nTraining time: {:.0f} seconds'.format(end_train - start))
print('\nPrediction time: {:.0f} seconds'.format(end_pred - end_train))
print('\nAccuracy: ', acc.sum()/acc.shape[0])

[LibSVM]
Training time: 261 seconds

Prediction time: 90 seconds

Accuracy:  0.8973
